In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [2]:
from models import MODELS_PATH

In [3]:
fasttext_model = f"{MODELS_PATH}/4m-oYYqjRvqVvyu2ryjmSw_trained_models/activity_FASTTEXT_CRAWL_SUB_SingleLayeredNN.pth"

In [4]:
from src.models import LogisticRegression, SingleLayeredNN

In [5]:
import torch

In [6]:
model = SingleLayeredNN(300, 300, 1)
model.load_state_dict(torch.load(fasttext_model))
model.eval()

SingleLayeredNN(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid2): Sigmoid()
)

In [13]:
import pandas as pd


df = pd.read_csv(f'{DATA_ROOT}/4m-oYYqjRvqVvyu2ryjmSw_common_words_100_final_data.csv')

In [14]:
from gensim.models import KeyedVectors


def _load_word_embedding_model(file=f'../models/fasttext/crawl-300d-2M-subword.vec', word_embedding_type="fasttext"):
    model = {}
    if file is None:
        file, *ign = embeddings.get("GLOVE_6B_300D")
    print("Loading Model")
    if word_embedding_type == "glove":
        df = pd.read_csv(file, sep=" ", quoting=3, header=None, index_col=0)
        model = {key: val.values for key, val in df.T.items()}
        print(len(model), " words loaded!")
    elif word_embedding_type == "word2vec":
        model = KeyedVectors.load_word2vec_format(file, binary=True)
    elif word_embedding_type == "fasttext":
        model = KeyedVectors.load_word2vec_format(file, binary=False)
    return model
MODEL = _load_word_embedding_model()

Loading Model


In [15]:
def _get_word_embeddings(word):
    try:
        return MODEL[word]
    except Exception as e:
        return None

In [16]:
df['Embedding'] = df["actual_words"].apply(_get_word_embeddings)

In [17]:
df.dropna(inplace=True)

In [18]:
x_data = df.loc[:, df.columns == 'Embedding']
y_data = df.loc[:, df.columns == 'activity']

In [21]:
from torch.autograd import Variable


# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(s_df):
    device = get_device()
    return Variable(torch.Tensor(s_df.values)).to(device)

def complex_df_to_tensor(_df):
    device = get_device()
    temp_x = []
    for index, row in _df.iterrows():
        temp_row = []
        row = row.to_dict()
        g_em = []
        for key in row.keys():
            if key != 'Embedding':
                temp_row.append(row[key])
            else:
                g_em = Variable(torch.Tensor(row[key])).to(device)
        temp_x.append(torch.cat([
            Variable(torch.Tensor(temp_row)).to(device), 
            g_em]
            ,dim=0))

    return torch.stack(temp_x, 0)

In [22]:
y_data = df_to_tensor(y_data)
x_data = complex_df_to_tensor(x_data)

<ipython-input-21-c151d4484f38>:28: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  g_em = Variable(torch.Tensor(row[key])).to(device)


In [23]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model.to(device)

SingleLayeredNN(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid2): Sigmoid()
)

In [24]:
from src.utils import evaluate


scores = evaluate(model, x_data, y_data)

In [25]:
str(scores["_accuracy"])

'0.9705804399681951'